Previously the setup pf3kgatk_callable_loci cleaned up output files after running. The output of this was used in sample metadata that was released, so we don't want to simply create a new setup based on this that doesn't remove the files, as we want to change parameters, and this will change released metadata. As such we will do this manually here. In the future, we should probably not do the cleanup so bed files are still available.

In [1]:
%run _standard_imports.ipynb
%run _shared_setup.ipynb


python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
import stat
from sh import ssh
bsub = sh.Command('bsub')

In [11]:
output_dir = "/lustre/scratch109/malaria/rp7/data/pf3k/pilot_5_0/20160525_CallableLoci_bed_release_5"
bam_fofn = "%s/pf3k_sample_bams.txt" % output_dir
!mkdir -p {output_dir}/scripts
!mkdir -p {output_dir}/results
!mkdir -p {output_dir}/logs


In [6]:
!vrpipe-fileinfo --setup pf3kgatk_mergelanes --step 4 --display tab --metadata sample > {bam_fofn}

In [7]:
GenomeAnalysisTK="/software/jre1.7.0_25/bin/java -Xmx4G -jar /nfs/team112_internal/production/tools/bin/gatk/GenomeAnalysisTK-3.5/GenomeAnalysisTK.jar"

In [10]:
GENOME_FN

'/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta'

In [9]:
tbl_bams = etl.fromtsv(bam_fofn)
print(len(tbl_bams.data()))
tbl_bams

2640


path,sample
/lustre/scratch109/malaria/pf3k_methods/output/0/8/5/a/291383/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PF0639-C
/lustre/scratch109/malaria/pf3k_methods/output/5/0/1/f/291199/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PF0360-C
/lustre/scratch109/malaria/pf3k_methods/output/b/0/8/3/292148/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PM0070-C
/lustre/scratch109/malaria/pf3k_methods/output/a/3/4/c/292075/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PH0938-C
/lustre/scratch109/malaria/pf3k_methods/output/c/7/e/3/291817/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PH0611-C


In [13]:
for bam_fn, sample in tbl_bams.data():
    print('.', sep='')
    bed_fn = "%s/results/callable_loci_%s.bed" % (output_dir, sample)
    summary_fn = "%s/results/summary_table_%s.txt" % (output_dir, sample)

    if not os.path.exists(bed_fn):
#     if True:
        script_fn = "%s/scripts/CallableLoci_%s.sh" % (output_dir, sample)
        fo = open(script_fn, 'w')
        print('''\
%s \
-T CallableLoci \
-R %s \
-I %s \
-summary %s \
-o %s
''' % (
                GenomeAnalysisTK,
                GENOME_FN,
                bam_fn,
                summary_fn,
                bed_fn,
            ),
            file = fo
        )
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/CL_%s.out' % (output_dir, sample),
            '-e', '%s/logs/CL_%s.err' % (output_dir, sample),
            '-J', 'CL_%s' % (sample),
            '-R', "'select[mem>8000] rusage[mem=8000]'",
            '-M', '8000',
            script_fn)


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [ ]:
2+2